In [ ]:
# pip install "hopsworks[python]"

import hopsworks
import pandas as pd
import numpy as np
import os

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Connect to Hopsworks
project = hopsworks.login(host="PROJECTURL",
                          port=PROJECTPORT,
                          api_key_value="KEY")

# Get the feature store
fs = project.get_feature_store()

In [ ]:
# 2. Generate the very important data
def generate_data(n_samples=1000):
    data = pd.DataFrame({
        'procrastination_level': np.random.randint(1, 11, n_samples),    # 1-10 scale
        'coffee_cups': np.random.randint(0, 8, n_samples),               # Number of coffee cups
        'last_minute_panic': np.random.randint(0, 2, n_samples),         # 1 for True, 0 for False
        'zodiac_sign': np.random.randint(1, 13, n_samples),              # Zodiac sign as integer (1-12)
        'task_completion': np.random.randint(0, 101, n_samples)          # Task completion as integer (0-100%)
    })
    return data

data = generate_data()

In [ ]:
# 3. Create feature group
fg = fs.get_or_create_feature_group(
    name='procrastinator_integer',
    version=1,
    primary_key=['procrastination_level', 'coffee_cups', 'last_minute_panic', 'zodiac_sign'],
    description='Features for predicting task completion based on procrastination',
    online_enabled=False
)
fg.insert(data, wait=True)

In [ ]:
# 4. Create a Feature view (mapping of the features for the model)
fv = fs.get_or_create_feature_view(
    name='procrastinator_integer_view2',
    version=1,
    description='Feature view for procrastination prediction',
    query=fg.select_all(),
    labels=['task_completion'],
)

In [ ]:
# 5. Get training data
X_train, X_test, y_train, y_test = fv.train_test_split(test_size=0.2)

In [ ]:
# 6. Train model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 7. Evaluate model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

In [ ]:
import os
from hsml.schema import Schema
from hsml.model_schema import ModelSchema
from joblib import dump

input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

# 8. Save model 
model_dir = "procrastinator_integers_model2"
os.makedirs(model_dir, exist_ok=True)

dump(model, os.path.join(model_dir, 'model.joblib'))

mr = project.get_model_registry()
procrastination_model = mr.sklearn.create_model(
    name="procrastinator_integers_model2",
    metrics={"mse": mse},
    model_schema=model_schema,
    input_example=X_test.sample().to_numpy(),
    description="Predicts task completion based on procrastination"
)
procrastination_model.save(model_dir)


In [ ]:
# 9. Create model serving endpoint
ms = project.get_model_serving()
deployment = procrastination_model.deploy(name="procrastinator40002k")

In [ ]:
deployment.start()

In [ ]:
# 10 Try to predict something.
input = {
    "instances": [
        [9,                     # procrastination_level
         10,                     # coffee_cups
         0,                     # last_minute_panic
         1                      # zodiac_sign
        ]  
    ]   
}

# Make the prediction
predictions = deployment.predict(input)
prediction_value = predictions['predictions'][0]

# Output the prediction result
print(f"Predicted task completion: {prediction_value:.2f}%")
